# 국민청원 분류하기 패치 Notebook

본 Notebook 파일은 기존 Notebook 파일을 실행 가능하도록 수정한 Notebook 입니다.

로컬 환경 Windows / Mac os 에서 테스트를 했으며 환경에 따른 모듈 문제로 인해 여러번의 시행착오가 있었습니다.

## 주의사항
1. 모듈간 의존성 문제로 오류가 발생하는 경우 새로운 가상환경을 만들어서 실행하세요.
2. torchtext 모듈 설치 시 환경에 맞는 설치 필요. conda / pip 환경 확인하세요.
3. konlpy 모듈은 JAVA 설치를 필요. 아래 konlpy 설치를 참고하세요.

## Install Dependencies

아래 모듈 설치는 새로운 가상 환경을 만든 후 새롭게 모듈을 설치 한 경우이며, 기존 환경에 설치하는 경우 명령어가 다를 수 있습니다.

[모듈 수동 설치]

설치 순서 :  pandas > numpy > konlpy > gensim > pytorch (반드시 torchtext 포함하여 설치)

만일 CPU 사용 환경이라면 다음 명령으로 pytorch 설치하세요.
```bash
# install basic modules
pip3 install pandas numpy konlpy gensim

# install pytorch 
# if non-conda env
pip3 install torch torchvision torchaudio torchtext

# if conda env
conda install pytorch torchvision torchaudio torchtext cpuonly -c pytorch

```

GPU를 사용하는 설치는 [Pytorch 사이트}(https://pytorch.org/)를 참고하세요.


[모듈 자동 설치]

개인별 환경이 다르므로 requirements 설치는 추천하지 않습니다. 설치된 버전 참고만 하시기 바랍니다.

```bash
# uninstall
pip list --format=freeze > installed-requirements.txt
pip uninstall -r installed-requirements.txt -y

# install for mac os
pip install -r requirements-patch.txt

# install for windows
pip install -r requirements-windows-only-cpu.txt
```

[konlpy 모듈 설치]
```bash 
 pip install konlpy
```

konlpy는 JAVA dll을 사용하므로 개별 환경에 맞는 JAVA를 설치하시기 바랍니다.
 - JAVA 1.8.x 버전 이상을 요구합니다.
 - 환경 별 설치
    - mac os : zulu-1.8.x 버전 설치. 테스트 완료.
    - windows : oralce 공식 jdk-17.0.12 버전 설치. 테스트 완료. 
    - Google colab은 [Konlpy 설치 링크](https://riverside13.tistory.com/entry/colab%EC%97%90-konlpy-%EC%84%A4%EC%B9%98%ED%95%98%EA%B8%B0#:~:text=colab%EC%97%90%20konlpy%20%EC%84%A4%EC%B9%98%ED%95%98%EA%B8%B0%201%201.%20bash%20%EC%85%B8%EB%A1%9C%20%EB%AA%85%EB%A0%B9%EC%96%B4%EB%A5%BC,%EC%84%A4%EC%B9%98%20%28%EC%8B%9C%EA%B0%84%20%EC%A2%80%20%EA%B1%B8%EB%A6%BC%29%204%204.%20%EB%8F%99%EC%9E%91%20%ED%99%95%EC%9D%B8) 참고. 테스트는 진행 하지 못함.
 - JAVA 설치 후 환경 변수 JAVA_HOME 설정이 필요함.
 - 단, JAVA_HOME 설정을 하지 않는 경우 소스 코드 내에서 jvm_path 정의 필요.(아래 코드 참고)

# 2.1 크롤링

국민 청원 사이트가 없어진 관계로 크롤링 영역은 삭제했습니다.

크롤링 코드는 원본 Notebook 에서 확인하세요.

크롤링 데이터는 `crawling.zip` 로 제공됩니다.

### 준비 사항

crawling.zip 압축 해제 : crawling.csv 파일은 ```05_국민청원_분류\data\``` 폴더 아래로 이동(Move) 시켜 주세요.

# 2.2 데이터 전처리

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np

# 저장된 csv 파일 읽기
df = pd.read_csv('data/crawling.csv')

In [2]:
df.loc[1]['content']  # 전처리 전

'우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저평가된 시장이라고 합니다. 하지만 투자매력이 없다고도 합니다.이렇게 말하는 이유가 어디 있습니까? 바로 투자를 해도 수익을 기대하기 어렵다는 인식이 이미 널리 퍼져 있다는 것입니다.\r\n\r\r\n지금은 투자매력이 없어서 그렇지,..우리 나라 시중 부동 자금이 어마어마 한 것으로 알려진 것과, 외국 투자 자본 또한 실로 어마무지하게 많다는 것도 알고있습니다. 그러나 이 투자금이 주식시장으로 원활하게 순환이 안되고 있다는데 있습니다.\r\n\r\r\n정부는 유휴자금이 주식 시장으로 들어오게 분위기를 띄어줘야 하는데,... 그렇지 못한 현실이 안타깝다고 생각합니다.\r\r\n국가가 지원해 돈 들이기가 어려우면,정부에서 정서적인 말이라도 활성화를 위한 관심표명과 정책적으로 지원만 해도 시장분위기는 많이 좋아질 것이라 확신합니다. \r\n\r\r\n그래서 저는 정부에 다음과 같이 청원합니다.\r\r\n주식시장에 더 큰 충격 오기 전에 정부가 예방 조치를 할 수 있는데까지 노력해 주시기를 당부 드리면서,...\r\n\r\r\n첫째,주식시장 활성화와 부양에 대한 정부의 의지를 표명해 주십시오 \r\n\r\r\n둘째,  코스닥 종목은 공매도 제외시켜 주세요.\r\n\r\r\n공매도 제도를 아예 없애버리면 좋겠지만 제도를 살린다면 적용 시장에서 코스닥 종목은 제외 해 주어야 체력이 약한 코스닥 시장을 안정시킬 수 있다고 봅니다 \r\n\r\r\n셋째, 거래시장의 주식거래세를 더 낮춰 주세요 \r\n\r\r\n특히 개미(소액)투자에 동일한 요율로 부과하는 것은 너무나 불공정 하다고 봅니다.\r\n\r\r\n넷째, 중,장기 보유자에 대한 우대 차원에서 보유기간을 설정하여 주세요.\r\n\r\r\n이는 장기 보유자에 대한 혜택을 주어 초단타매매 등 시장 질서를 교란하는 투자자와 분리하는 제도를 도입해 시장 안정화에 기여해 주시길 간곡히 부탁드리면서 청원합니다.'

[전처리]

In [3]:
import re

def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [4]:
df.loc[1]['content']  # 전처리 후

'우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저평가된 시장이라고 합니다  하지만 투자매력이 없다고도 합니다 이렇게 말하는 이유가 어디 있습니까  바로 투자를 해도 수익을 기대하기 어렵다는 인식이 이미 널리 퍼져 있다는 것입니다      지금은 투자매력이 없어서 그렇지 우리 나라 시중 부동 자금이 어마어마 한 것으로 알려진 것과  외국 투자 자본 또한 실로 어마무지하게 많다는 것도 알고있습니다  그러나 이 투자금이 주식시장으로 원활하게 순환이 안되고 있다는데 있습니다      정부는 유휴자금이 주식 시장으로 들어오게 분위기를 띄어줘야 하는데  그렇지 못한 현실이 안타깝다고 생각합니다    국가가 지원해 돈 들이기가 어려우면 정부에서 정서적인 말이라도 활성화를 위한 관심표명과 정책적으로 지원만 해도 시장분위기는 많이 좋아질 것이라 확신합니다       그래서 저는 정부에 다음과 같이 청원합니다    주식시장에 더 큰 충격 오기 전에 정부가 예방 조치를 할 수 있는데까지 노력해 주시기를 당부 드리면서      첫째 주식시장 활성화와 부양에 대한 정부의 의지를 표명해 주십시오      둘째   코스닥 종목은 공매도 제외시켜 주세요      공매도 제도를 아예 없애버리면 좋겠지만 제도를 살린다면 적용 시장에서 코스닥 종목은 제외 해 주어야 체력이 약한 코스닥 시장을 안정시킬 수 있다고 봅니다      셋째  거래시장의 주식거래세를 더 낮춰 주세요      특히 개미 소액 투자에 동일한 요율로 부과하는 것은 너무나 불공정 하다고 봅니다      넷째  중 장기 보유자에 대한 우대 차원에서 보유기간을 설정하여 주세요      이는 장기 보유자에 대한 혜택을 주어 초단타매매 등 시장 질서를 교란하는 투자자와 분리하는 제도를 도입해 시장 안정화에 기여해 주시길 간곡히 부탁드리면서 청원합니다 '

# 2.3 토크나이징 및 변수 생성

[토크나이징]

In [5]:
from konlpy.tag import Okt
import platform

# konlpy 에서 JAVA 필요함.
# JVM 1.8 버전 이상
try:
    # JAVA_HOME 설정이 된 경우
    # JAVA_HOME 설정이 되더라도 dll을 찾지 못해 오류가 발생할 수 있음.
    okt = Okt()
except:
    # JAVA_HOME 설정이 없는 경우 직접 설정
    print('JAVA_HOME not found.')
    
    # check platform
    platform_name = platform.system()
    print(f'Platform name : {platform_name}')

    if platform_name=='Darwin': # for mac os
        # JVM_PATH konlpy 가 요구하는 dylib 파일 경로로 셋팅.
        # mac os의 경우 JDK 설치 후 설치된 폴더명을 1.8.0을 포함하도록 변경 필요함. ex) zulu-1.8.jdk -> zulu-1.8.0.jdk
        # konlpy 가 요구하는 libjli.dylib 파일 경로로 셋팅.
        JVM_PATH = '/Library/Java/JavaVirtualMachines/zulu-1.8.0.jdk/Contents/Home/jre/lib/jli/libjli.dylib'
    else:
        # konlpy 가 요구하는 jvm.dll 파일 경로로 셋팅.
        JVM_PATH = 'C:\\Library\\Java\\jdk-17.0.12\\bin\\server\\jvm.dll'
    okt = Okt(jvmpath=JVM_PATH)

df['title_token'] = df.title.apply(okt.morphs)
print('title_token completed.')
df['content_token'] = df.content.apply(okt.nouns)
print('content_token completed.')

JAVA_HOME not found.
Platform name : Windows
title_token completed.
content_token completed.


[파생변수 생성]

In [6]:
df['token_final'] = df.title_token + df.content_token

df['count'] = df['count'].replace({',' : ''}, regex = True).apply(lambda x : int(x))

print(df.dtypes)

df['label'] = df['count'].apply(lambda x: 'Yes' if x>=1000 else 'No')

category         object
content          object
count             int64
end              object
start            object
title            object
title_token      object
content_token    object
token_final      object
dtype: object


In [7]:
df_drop = df[['token_final', 'label']]

In [8]:
df_drop.head()

,token_final,label
0,"[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,...",No
1,"[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호, 우리, 나라, 코스피, 총...",No
2,"[교정, 기관, 의, 민낮, 일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이...",No
3,"[미세먼지, 저, 감, 대책, 미세먼지, 심각, 성은, 이제, 적극, 대안, 요구,...",No
4,"[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ...",Yes


[데이터 엑셀로 저장]

In [9]:
df_drop.to_csv('data/df_drop.csv', index = False, encoding = 'utf-8-sig')

# 2.4 단어 임베딩

[단어 임베딩]

In [10]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(df_drop['token_final'], 
                           sg = 1, # skip-gram
                           vector_size = 100, 
                           window = 2, 
                           min_count = 1, 
                           workers = 4
                           )

print(embedding_model)

model_result = embedding_model.wv.most_similar("음주운전")
print(model_result)

Word2Vec<vocab=42642, vector_size=100, alpha=0.025>
[('음주', 0.8596104383468628), ('뺑소니', 0.839146077632904), ('무면허', 0.8311192393302917), ('살인죄', 0.7677296996116638), ('살인자', 0.7575804591178894), ('윤창', 0.7524190545082092), ('전과자', 0.742548942565918), ('형량', 0.7399409413337708), ('촉법소년', 0.7383947372436523), ('엄하', 0.7378968000411987)]


[임베딩 모델 저장 및 로드]

In [11]:
from gensim.models import KeyedVectors

embedding_model.wv.save_word2vec_format('data/petitions_tokens_w2v.pkl') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('data/petitions_tokens_w2v.pkl') # 모델 로드

model_result = loaded_model.most_similar("음주운전")
print(model_result)

[('음주', 0.8596104383468628), ('뺑소니', 0.839146077632904), ('무면허', 0.8311192393302917), ('살인죄', 0.7677296996116638), ('살인자', 0.7575804591178894), ('윤창', 0.7524190545082092), ('전과자', 0.742548942565918), ('형량', 0.7399409413337708), ('촉법소년', 0.7383947372436523), ('엄하', 0.7378968000411987)]


# 2.5 실험 설계

[데이터셋 분할 및 저장]

In [12]:
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('data/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('data/validation.csv', index=False, encoding='utf-8-sig')

[Field클래스 정의]

In [13]:
import torchtext
from torchtext.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential = False)

[데이터 불러오기]

In [14]:
from torchtext.data import TabularDataset

train, validation = TabularDataset.splits(
    path = 'data/',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)

print("Train:", train[0].text,  train[0].label)
print("Validation:", validation[0].text, validation[0].label)

Train: ['유부', '남', '유부녀', '의', '미혼', '빙자', '에', '대한', '처벌', '이나', '대안', '이', '필요합니다', '사람', '만남', '개인', '사항', '생각', '영화', '소설', '속', '일이', '생각', '작년', '교재', '남자', '첫', '만남', '나이', '결혼', '생각', '미혼', '서로', '감정', '쌍둥이', '자연', '임신', '사실', '때', '바로', '이별', '말', '뒤', '말', '못', '유전병', '그', '아픔', '애', '결혼', '생각', '다음', '만날', '때', '타지', '파견', '근무', '혼인신고', '것', '부모님', '결혼', '반대', '며', '집안', '점', '여자', '안보', '저', '결혼', '반대', '설득', '시간', '또', '시간', '대뜸', '사실', '결혼', '처음', '서로', '결혼', '자주', '양가', '부모님', '혼반', '서로', '일', '터치', '연락', '살기', '별거', '및', '왕래', '즉', '서류', '유부', '남', '애', '별거', '이상', '요', '충격', '쌍둥이', '아빠', '이혼', '생각', '물음', '이혼', '답', '애도', '둘', '사이', '연락', '저', '사이', '전', '불륜', '녀', '혼외자', '사람', '매일', '울', '충격', '생명', '둘', '눈총', '게', '감수', '생각', '오늘', '사실', '딸', '부모님', '부인', '집안', '행사', '부모님', '참여', '왕래', '이혼', '딸', '노력', '지금', '시간', '온', '상투', '말로', '저', '제', '쌍둥이', '비수', '꽃았습니', '유부', '남', '애도', '저희', '가족', '친척', '일터', '사람', '지인', '지금', '시간', '지방법원', '무료', '변호사', '처지', '답', '법률', '사무소

[단어장 및 DataLoader 정의]

In [15]:
import torch
from torchtext.vocab import Vectors
from torchtext.data import BucketIterator

vectors = Vectors(name="data/petitions_tokens_w2v.pkl")

TEXT.build_vocab(train, vectors = vectors, min_freq = 1, max_size = None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

c:\Users\jeong\anaconda3\envs\petitions-cpu-env\lib\site-packages\torchtext\vocab.py:432: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.itos, self.stoi, self.vectors, s

임베딩 벡터의 개수와 차원 : torch.Size([39118, 100]) 


# 2.6 TextCNN

[TextCNN 모델링]

In [16]:
import torch.nn as nn   
import torch.optim as optim 
import torch.nn.functional as F 

class TextCNN(nn.Module): 
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)      
    
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()                
        self.dropout = nn.Dropout(0.4)         
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)     
        
    def forward(self, x):  
      
        emb_x = self.embed(x)           
        emb_x = emb_x.unsqueeze(1)  

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]       

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]    
        
        fc_x = torch.cat(pool_x, dim=1) 
        fc_x = fc_x.squeeze(-1)       
        fc_x = self.dropout(fc_x)         

        logit = self.fc(fc_x)     
        
        return logit

[모델 학습 함수 정의]

In [17]:
def train(model, device, train_itr, optimizer):
    
    model.train()                               
    corrects, train_loss = 0.0,0        
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label      
        text = torch.transpose(text, 0, 1)          
        target.data.sub_(1)                                 
        text, target = text.to(device), target.to(device)  

        optimizer.zero_grad()                           
        logit = model(text)                         
    
        loss = F.cross_entropy(logit, target)   
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()    
        result = torch.max(logit,1)[1] 
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

[모델 평가 함수 정의]

In [18]:
def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:
        
        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset) 
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

[모델 학습 및 성능 확인]

In [19]:
model = TextCNN(vocab, 100, 10, [3, 4, 5], 2).to(device)
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1, 3+1):
 
    tr_loss, tr_acc = train(model, device, train_iter, optimizer) 
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))
        
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
    
    print('-----------------------------------------------------------------------------')

TextCNN(
  (embed): Embedding(39118, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
Train Epoch: 1 	 Loss: 0.08421617247194206 	 Accuracy: 58.63296890258789%
Valid Epoch: 1 	 Loss: 0.08223568242700662 	 Accuracy: 61.25918960571289%
model saves at 61.25918960571289 accuracy
-----------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.07749072197249127 	 Accuracy: 66.24928283691406%
Valid Epoch: 2 	 Loss: 0.07875609837527223 	 Accuracy: 65.3492660522461%
model saves at 65.3492660522461 accuracy
-----------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.06003378599385706 	 Accuracy: 77.93222045898438%
Valid Epoch: 3 	 Loss: 